In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter
import optuna as op

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
#URM_train, URM_val = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")

In [6]:
evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1008 ( 2.8%) Users that have less than 1 test interactions


In [7]:
from libs.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [8]:
def objective_function_KNN(optuna_trial):
                       
    similarity="tversky"
    
    recommender_instance = ItemKNNCFRecommender(URM_train)
    
    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 2, 50),
                   "shrink": optuna_trial.suggest_float("shrink", 1 , 25),
                   "feature_weighting": optuna_trial.suggest_categorical("feature_weighting", ["BM25", "TF-IDF", "none"]),
                   "normalize": optuna_trial.suggest_categorical("normalize", [True, False])
                  }
    
    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0.1, 0.99, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0.1, 1.9, log=False)
        full_hyperp["normalize"] = True 

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])
        
    
    recommender_instance.fit(**full_hyperp)
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)   
    
    # Save MAP alongside RECALL
    trial_results = {
        "RECALL": eval_res["RECALL"][10],
        "MAP": eval_res["MAP"][10]  # Save MAP for reference
    }
    
    # Save the MAP value to the optuna trial user attributes for later use
    optuna_trial.set_user_attr("MAP", trial_results["MAP"])
    
    # Return RECALL for optimization
    return trial_results["RECALL"]

In [9]:
class SaveResults(object):
    def __init__(self, file_path=None):
        self.file_path = file_path
        self.results = []
    
    def __call__(self, optuna_study, optuna_trial):
        trial_result = optuna_trial.params.copy()
        trial_result["iteration"] = optuna_trial.number
        trial_result["RECALL"] = optuna_trial.value
        
        # Retrieve the MAP metric from user attributes
        trial_result["MAP"] = optuna_trial.user_attrs.get("MAP")
        
        self.results.append(trial_result)
        
        # Convert to DataFrame and save to CSV
        results_df = pd.DataFrame(self.results)
        results_df.to_csv(self.file_path, index=False)


In [11]:
optuna_study = op.create_study(direction="maximize")
        
save_results_tanimoto = SaveResults("result_experiments/ItemKNNCFRecommender/results_tversky_optuna_recall.csv")
        
optuna_study.optimize(objective_function_KNN,
                      callbacks=[save_results_tanimoto],
                      n_trials = 400)

[I 2024-12-26 01:13:50,931] A new study created in memory with name: no-name-4ac80043-f512-475c-88f7-2a6781cbeb97


Similarity column 38121 (100.0%), 5094.61 column/sec. Elapsed time 7.48 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 8.66 sec. Users per second: 4009


[I 2024-12-26 01:14:07,223] Trial 0 finished with value: 0.1092142859885148 and parameters: {'topK': 9, 'shrink': 24.089285247117747, 'feature_weighting': 'BM25', 'normalize': True, 'tversky_alpha': 0.23395549947699276, 'tversky_beta': 1.3434575160393993}. Best is trial 0 with value: 0.1092142859885148.


Similarity column 38121 (100.0%), 5108.64 column/sec. Elapsed time 7.46 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 8.75 sec. Users per second: 3969


[I 2024-12-26 01:14:23,547] Trial 1 finished with value: 0.11367735561015511 and parameters: {'topK': 5, 'shrink': 15.223312050813266, 'feature_weighting': 'none', 'normalize': False, 'tversky_alpha': 0.9181056608982311, 'tversky_beta': 0.7736133317488086}. Best is trial 1 with value: 0.11367735561015511.


...

[I 2024-12-26 03:13:31,070] Trial 397 finished with value: 0.11713153359009996 and parameters: {'topK': 3, 'shrink': 23.822285534598688, 'feature_weighting': 'TF-IDF', 'normalize': False, 'tversky_alpha': 0.14589552191993113, 'tversky_beta': 1.6957760480842856}. Best is trial 281 with value: 0.11874846200526792.


Similarity column 38121 (100.0%), 5326.93 column/sec. Elapsed time 7.16 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 8.20 sec. Users per second: 4235


[I 2024-12-26 03:13:46,564] Trial 398 finished with value: 0.11842205410935974 and parameters: {'topK': 5, 'shrink': 22.034718519239668, 'feature_weighting': 'TF-IDF', 'normalize': False, 'tversky_alpha': 0.3274519540362769, 'tversky_beta': 1.8019344321457857}. Best is trial 281 with value: 0.11874846200526792.


Similarity column 38121 (100.0%), 5323.23 column/sec. Elapsed time 7.16 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 8.16 sec. Users per second: 4258


[I 2024-12-26 03:14:02,021] Trial 399 finished with value: 0.10721877532469669 and parameters: {'topK': 4, 'shrink': 20.47049282776635, 'feature_weighting': 'BM25', 'normalize': False, 'tversky_alpha': 0.10895234557741798, 'tversky_beta': 1.861708540293437}. Best is trial 281 with value: 0.11874846200526792.


In [15]:
pd.DataFrame(save_results_tanimoto.results).sort_values(by=["MAP"], ascending=False)

,topK,shrink,feature_weighting,normalize,tversky_alpha,tversky_beta,iteration,RECALL,MAP
96,6,21.589349,TF-IDF,False,0.167252,1.722013,96,0.118551,0.028130
220,6,22.217756,none,False,0.227589,1.788536,220,0.118239,0.028126
102,6,21.079349,TF-IDF,False,0.196247,1.733155,102,0.118240,0.028112
83,6,22.622420,TF-IDF,False,0.255611,1.770004,83,0.118308,0.028104
249,6,20.613082,none,False,0.235928,1.767350,249,0.118309,0.028103
...,...,...,...,...,...,...,...,...,...
6,19,18.287544,BM25,True,0.702835,0.703088,6,0.103331,0.023864
5,47,8.640345,BM25,False,0.346342,1.396634,5,0.102509,0.023736
3,42,8.196521,BM25,False,0.173447,0.219287,3,0.099767,0.022978
43,2,22.286228,TF-IDF,False,0.320202,0.102723,43,0.100111,0.022212


In [18]:
save_results_tanimoto.results[96]

{'topK': 6,
 'shrink': 21.589349496863413,
 'feature_weighting': 'TF-IDF',
 'normalize': False,
 'tversky_alpha': 0.16725184866550824,
 'tversky_beta': 1.722012798691049,
 'iteration': 96,
 'RECALL': 0.11855115150555026,
 'MAP': 0.02812995231879061}

In [18]:
save_results_cosine.results_df['result'].max() 

0.0263499559387598

In [33]:
save_results_adjusted.results_df.to_csv("result_experiments/ItemKNNCFRecommender/results_adjusted.csv")